# Create a SageMaker MLOps Project for Pipelines
Automatically run pipelines when code changes using Amazon SageMaker Projects

Note:  This requires that you have enabled products within SageMaker Studio

![](../img/enable-service-catalog-portfolio-for-studio.png)

In [1]:
import os
import sagemaker
import logging
import boto3
import sagemaker
import pandas as pd
from pprint import pprint

sess = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name

sm = boto3.Session().client(service_name="sagemaker", region_name=region)
sc = boto3.Session().client(service_name="servicecatalog", region_name=region)
sts = boto3.Session().client(service_name="sts", region_name=region)
iam = boto3.Session().client(service_name="iam", region_name=region)
codepipeline = boto3.Session().client("codepipeline", region_name=region)

In [2]:
search_response = sc.search_products(
    Filters={"FullTextSearch": ["MLOps template for model building, training, and deployment"]}
)

sagemaker_pipeline_product_id = search_response["ProductViewSummaries"][0]["ProductId"]
print(sagemaker_pipeline_product_id)

prod-j3ufw6hl7utxm


In [3]:
describe_response = sc.describe_product(Id=sagemaker_pipeline_product_id)

sagemaker_pipeline_product_provisioning_artifact_id = describe_response["ProvisioningArtifacts"][0]["Id"]

In [4]:
print(sagemaker_pipeline_product_provisioning_artifact_id)

pa-oacphmo7m2bji


# Create a SageMaker Project


In [5]:
import time

timestamp = int(time.time())

In [6]:
sagemaker_project_name = "dsoaws-{}".format(timestamp)

create_response = sm.create_project(
    ProjectName=sagemaker_project_name,
    ProjectDescription="dsoaws-{}".format(timestamp),
    ServiceCatalogProvisioningDetails={
        "ProductId": sagemaker_pipeline_product_id,
        "ProvisioningArtifactId": sagemaker_pipeline_product_provisioning_artifact_id,
    },
)

sagemaker_project_id = create_response["ProjectId"]
sagemaker_project_arn = create_response["ProjectArn"]

print("Project ID {}".format(sagemaker_project_id))
print("Project ARN {}".format(sagemaker_project_arn))

Project ID p-nynqfqk1cz5v
Project ARN arn:aws:sagemaker:us-east-1:231218423789:project/dsoaws-1615231417


In [7]:
sagemaker_project_name_and_id = "{}-{}".format(sagemaker_project_name, sagemaker_project_id)

print("Combined Project ID and ARN combined: {}".format(sagemaker_project_name_and_id))

Combined Project ID and ARN combined: dsoaws-1615231417-p-nynqfqk1cz5v


# _Wait for the Project to be Created_

In [8]:
%%time

import time

try:
    describe_project_response = sm.describe_project(ProjectName=sagemaker_project_name)
    project_status = describe_project_response["ProjectStatus"]
    print("Creating Project...")

    while project_status in ["Pending", "CreateInProgress"]:
        print("Please wait...")
        time.sleep(30)
        describe_project_response = sm.describe_project(ProjectName=sagemaker_project_name)
        project_status = describe_project_response["ProjectStatus"]
        print("Project status: {}".format(project_status))

    if project_status == "CreateCompleted":
        print("Project {}".format(project_status))

    else:
        print("Project status: {}".format(project_status))
        raise Exception("Project not created.")

except Exception as e:
    print(e)

print(describe_project_response)

Creating Project...
Please wait...
Project status: CreateInProgress
Please wait...
Project status: CreateInProgress
Please wait...
Project status: CreateInProgress
Please wait...
Project status: CreateInProgress
Please wait...
Project status: CreateInProgress
Please wait...
Project status: CreateCompleted
Project CreateCompleted
{'ProjectArn': 'arn:aws:sagemaker:us-east-1:231218423789:project/dsoaws-1615231417', 'ProjectName': 'dsoaws-1615231417', 'ProjectId': 'p-nynqfqk1cz5v', 'ProjectDescription': 'dsoaws-1615231417', 'ServiceCatalogProvisioningDetails': {'ProductId': 'prod-j3ufw6hl7utxm', 'ProvisioningArtifactId': 'pa-oacphmo7m2bji'}, 'ServiceCatalogProvisionedProductDetails': {'ProvisionedProductId': 'pp-jx3cgosqxbvhc'}, 'ProjectStatus': 'CreateCompleted', 'CreatedBy': {'UserProfileArn': 'arn:aws:sagemaker:us-east-1:231218423789:user-profile/d-scapt1dubpfo/demo', 'UserProfileName': 'demo', 'DomainId': 'd-scapt1dubpfo'}, 'CreationTime': datetime.datetime(2021, 3, 8, 19, 23, 37, 7090

# _Wait for Project to be Created ^^ Above ^^_

# Attach IAM Policies for FeatureStore 
This is the role used by Code Build when it starts the pipeline.

In [9]:
sc_role_name = "AmazonSageMakerServiceCatalogProductsUseRole"

In [10]:
account_id = sts.get_caller_identity()["Account"]
print(account_id)

231218423789


In [11]:
response = iam.attach_role_policy(RoleName=sc_role_name, PolicyArn="arn:aws:iam::aws:policy/AmazonSageMakerFullAccess")

print(response)

{'ResponseMetadata': {'RequestId': '585947e3-0018-4b20-87c5-acb920ca6d2a', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '585947e3-0018-4b20-87c5-acb920ca6d2a', 'content-type': 'text/xml', 'content-length': '212', 'date': 'Mon, 08 Mar 2021 19:26:38 GMT'}, 'RetryAttempts': 0}}


In [12]:
response = iam.attach_role_policy(
    RoleName=sc_role_name, PolicyArn="arn:aws:iam::aws:policy/AmazonSageMakerFeatureStoreAccess"
)

print(response)

{'ResponseMetadata': {'RequestId': 'b89a99b9-7bed-4786-97f0-d87c11810323', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': 'b89a99b9-7bed-4786-97f0-d87c11810323', 'content-type': 'text/xml', 'content-length': '212', 'date': 'Mon, 08 Mar 2021 19:26:38 GMT'}, 'RetryAttempts': 0}}


In [13]:
response = iam.attach_role_policy(RoleName=sc_role_name, PolicyArn="arn:aws:iam::aws:policy/IAMFullAccess")

print(response)

{'ResponseMetadata': {'RequestId': '98bca404-7b84-40b0-83bd-f52e2c559bb7', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '98bca404-7b84-40b0-83bd-f52e2c559bb7', 'content-type': 'text/xml', 'content-length': '212', 'date': 'Mon, 08 Mar 2021 19:26:38 GMT'}, 'RetryAttempts': 0}}


# Stop the `Abalone` Sample Pipeline that Ships with SageMaker Pipelines
The sample "abalone" pipeline starts automatically when we create the project.  We want to stop this pipeline to release these resources and use them for our own pipeline.

In [15]:
# sample_abalone_pipeline_execution_arn = sm.list_pipeline_executions(PipelineName=sagemaker_project_name_and_id)["PipelineExecutionSummaries"][0]["PipelineExecutionArn"]

# print(sample_abalone_pipeline_execution_arn)

In [ ]:
# sm.stop_pipeline_execution(PipelineExecutionArn=sample_abalone_pipeline_execution_arn)

In [ ]:
# %%time

# try:
#     describe_pipeline_execution_response = sm.describe_pipeline_execution(
#         PipelineExecutionArn=sample_abalone_pipeline_execution_arn
#     )
#     pipeline_execution_status = describe_pipeline_execution_response["PipelineExecutionStatus"]

#     while pipeline_execution_status not in ["Stopped", "Failed"]:
#         print("Please wait...")
#         time.sleep(30)
#         describe_pipeline_execution_response = sm.describe_pipeline_execution(
#             PipelineExecutionArn=sample_abalone_pipeline_execution_arn
#         )
#         pipeline_execution_status = describe_pipeline_execution_response["PipelineExecutionStatus"]
#         print("Pipeline execution status: {}".format(pipeline_execution_status))

#     if pipeline_execution_status in ["Stopped", "Failed"]:
#         print("Pipeline execution status {}".format(pipeline_execution_status))
#     else:
#         print("Pipeline execution status: {}".format(pipeline_execution_status))
#         raise Exception("Pipeline execution not deleted.")

# except Exception as e:
#     print(e)

# print(describe_pipeline_execution_response)

In [ ]:
# sm.delete_pipeline(PipelineName=sagemaker_project_name_and_id)

# Clone the MLOps Repositories in AWS CodeCommit

In [16]:
import os

sm_studio_root_path = "/root/"
sm_notebooks_root_path = "/home/ec2-user/SageMaker/"

root_path = sm_notebooks_root_path if os.path.isdir(sm_notebooks_root_path) else sm_studio_root_path

print(root_path)

/root/


In [17]:
print(region)

us-east-1


In [18]:
code_commit_repo1 = "https://git-codecommit.{}.amazonaws.com/v1/repos/sagemaker-{}-modelbuild".format(
    region, sagemaker_project_name_and_id
)
print(code_commit_repo1)

https://git-codecommit.us-east-1.amazonaws.com/v1/repos/sagemaker-dsoaws-1615231417-p-nynqfqk1cz5v-modelbuild


In [19]:
sagemaker_mlops_build_code = "{}{}/sagemaker-{}-modelbuild".format(
    root_path, sagemaker_project_name_and_id, sagemaker_project_name_and_id
)
print(sagemaker_mlops_build_code)

/root/dsoaws-1615231417-p-nynqfqk1cz5v/sagemaker-dsoaws-1615231417-p-nynqfqk1cz5v-modelbuild


In [20]:
code_commit_repo2 = "https://git-codecommit.{}.amazonaws.com/v1/repos/sagemaker-{}-modeldeploy".format(
    region, sagemaker_project_name_and_id
)
print(code_commit_repo2)

https://git-codecommit.us-east-1.amazonaws.com/v1/repos/sagemaker-dsoaws-1615231417-p-nynqfqk1cz5v-modeldeploy


In [21]:
sagemaker_mlops_deploy_code = "{}{}/sagemaker-{}-modeldeploy".format(
    root_path, sagemaker_project_name_and_id, sagemaker_project_name_and_id
)
print(sagemaker_mlops_deploy_code)

/root/dsoaws-1615231417-p-nynqfqk1cz5v/sagemaker-dsoaws-1615231417-p-nynqfqk1cz5v-modeldeploy


In [22]:
!git config --global credential.helper '!aws codecommit credential-helper $@'
!git config --global credential.UseHttpPath true

In [23]:
!git clone $code_commit_repo1 $sagemaker_mlops_build_code

Cloning into '/root/dsoaws-1615231417-p-nynqfqk1cz5v/sagemaker-dsoaws-1615231417-p-nynqfqk1cz5v-modelbuild'...
remote: Counting objects: 26, done.
Unpacking objects: 100% (26/26), done.


In [24]:
!git clone $code_commit_repo2 $sagemaker_mlops_deploy_code

Cloning into '/root/dsoaws-1615231417-p-nynqfqk1cz5v/sagemaker-dsoaws-1615231417-p-nynqfqk1cz5v-modeldeploy'...
remote: Counting objects: 12, done.
Unpacking objects: 100% (12/12), done.


# Remove Sample `Abalone` Example Code

In [25]:
!rm -rf $sagemaker_mlops_build_code/pipelines/abalone

# Copy Workshop Code Into Local Project Folders

In [26]:
workshop_project_build_code = "{}workshop/10_pipeline/sagemaker_mlops/sagemaker-project-modelbuild".format(root_path)
print(workshop_project_build_code)

/root/workshop/10_pipeline/sagemaker_mlops/sagemaker-project-modelbuild


In [27]:
workshop_project_deploy_code = "{}workshop/10_pipeline/sagemaker_mlops/sagemaker-project-modeldeploy".format(root_path)
print(workshop_project_deploy_code)

/root/workshop/10_pipeline/sagemaker_mlops/sagemaker-project-modeldeploy


In [28]:
!cp -R $workshop_project_build_code/* $sagemaker_mlops_build_code/

In [29]:
!cp -R $workshop_project_deploy_code/* $sagemaker_mlops_deploy_code/

# Commit New Code 

In [30]:
print(sagemaker_mlops_build_code)

/root/dsoaws-1615231417-p-nynqfqk1cz5v/sagemaker-dsoaws-1615231417-p-nynqfqk1cz5v-modelbuild


In [31]:
!cd $sagemaker_mlops_build_code; git status; git add --all .; git commit -m "Data Science on AWS"; git push

On branch main
Your branch is up to date with 'origin/main'.

Changes not staged for commit:
  (use "git add/rm <file>..." to update what will be committed)
  (use "git checkout -- <file>..." to discard changes in working directory)

	modified:   README.md
	modified:   codebuild-buildspec.yml
	deleted:    pipelines/abalone/__init__.py
	deleted:    pipelines/abalone/evaluate.py
	deleted:    pipelines/abalone/pipeline.py
	deleted:    pipelines/abalone/preprocess.py
	modified:   pipelines/run_pipeline.py
	modified:   setup.py

Untracked files:
  (use "git add <file>..." to include in what will be committed)

	pipelines/dsoaws/

no changes added to commit (use "git add" and/or "git commit -a")
[main d7b7e8b] Data Science on AWS
 14 files changed, 2197 insertions(+), 456 deletions(-)
 delete mode 100644 pipelines/abalone/evaluate.py
 delete mode 100644 pipelines/abalone/pipeline.py
 delete mode 100644 pipelines/abalone/preprocess.py
 rename pipelines/{abalone => dsoaws}/__init__.py (100%)
 

In [32]:
!cd $sagemaker_mlops_deploy_code; git status; git add --all .; git commit -m "Data Science on AWS"; git push

On branch main
Your branch is up to date with 'origin/main'.

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git checkout -- <file>..." to discard changes in working directory)

	modified:   README.md
	modified:   prod-config.json
	modified:   staging-config.json
	modified:   test/test.py

no changes added to commit (use "git add" and/or "git commit -a")
[main e0c4217] Data Science on AWS
 4 files changed, 4 insertions(+), 6 deletions(-)
Enumerating objects: 13, done.
Counting objects: 100% (13/13), done.
Delta compression using up to 2 threads
Compressing objects: 100% (7/7), done.
Writing objects: 100% (7/7), 617 bytes | 88.00 KiB/s, done.
Total 7 (delta 5), reused 0 (delta 0)
To https://git-codecommit.us-east-1.amazonaws.com/v1/repos/sagemaker-dsoaws-1615231417-p-nynqfqk1cz5v-modeldeploy
   ef5ed27..e0c4217  main -> main


# Store the Variables

In [33]:
%store sagemaker_mlops_build_code
%store sagemaker_mlops_deploy_code
%store sagemaker_project_name
%store sagemaker_project_id
%store sagemaker_project_name_and_id
%store sagemaker_project_arn
%store sagemaker_pipeline_product_id
%store sagemaker_pipeline_product_provisioning_artifact_id

Stored 'sagemaker_mlops_build_code' (str)
Stored 'sagemaker_mlops_deploy_code' (str)
Stored 'sagemaker_project_name' (str)
Stored 'sagemaker_project_id' (str)
Stored 'sagemaker_project_name_and_id' (str)
Stored 'sagemaker_project_arn' (str)
Stored 'sagemaker_pipeline_product_id' (str)
Stored 'sagemaker_pipeline_product_provisioning_artifact_id' (str)


In [34]:
!ls -al $sagemaker_mlops_build_code/pipelines/dsoaws/

total 100
drwxr-xr-x 3 root root  6144 Mar  8 19:29 .
drwxr-xr-x 3 root root  6144 Mar  8 19:29 ..
-rw-r--r-- 1 root root     0 Mar  8 19:29 __init__.py
-rw-r--r-- 1 root root  8039 Mar  8 19:29 evaluate_model_metrics.py
-rw-r--r-- 1 root root  3501 Mar  8 19:29 inference.py
-rw-r--r-- 1 root root 16552 Mar  8 19:29 pipeline.py
-rw-r--r-- 1 root root 22207 Mar  8 19:29 preprocess-scikit-text-to-bert-feature-store.py
drwxr-xr-x 2 root root  6144 Mar  8 19:29 test_data
-rw-r--r-- 1 root root 25080 Mar  8 19:29 tf_bert_reviews.py


In [35]:
!pygmentize $sagemaker_mlops_build_code/pipelines/dsoaws/pipeline.py

"""
Example workflow pipeline script for BERT pipeline.

                                                 . -RegisterModel
                                                .
    Process-> Train -> (Evaluate -> Condition) .
                                                .
                                                 . -(stop)

Implements a get_pipeline(**kwargs) method.
"""

import os
import boto3
import logging
import time

from botocore.exceptions import ClientError

import sagemaker
import sagemaker.session
from sagemaker.estimator import Estimator
from sagemaker.inputs import TrainingInput
from sagemaker.sklearn.processing import SKLearnProcessor
from sagemaker.tensorflow import TensorFlow

from sagemaker.model_metrics import (
    MetricsSource,
    ModelMetrics,
)

from sagemaker.processing import (
    ProcessingInput,
    ProcessingOutput,
    ScriptProcessor,
)

from sagemaker.workflow.parameters import ParameterInteger, ParameterString, ParameterFloat

from sagemaker.workf

# Wait for Pipeline Execution to Start
Now that we have committed code, our pipeline will start.  Let's wait for the pipeline to start.


# TODO: Check Why Pipeline Doesn't Start

In [36]:
%%time

import time
from pprint import pprint

while True:
    try:
        print("Listing executions for our pipeline...")
        list_executions_response = sm.list_pipeline_executions(PipelineName=sagemaker_project_name_and_id)[
            "PipelineExecutionSummaries"
        ]
        break
    except Exception as e:
        print("Please wait...")
        time.sleep(30)

pprint(list_executions_response)

Listing executions for our pipeline...
Please wait...
Listing executions for our pipeline...
Please wait...
Listing executions for our pipeline...
Please wait...
Listing executions for our pipeline...
Please wait...
Listing executions for our pipeline...
Please wait...
Listing executions for our pipeline...
Please wait...
Listing executions for our pipeline...
Please wait...
Listing executions for our pipeline...
Please wait...
Listing executions for our pipeline...
Please wait...
Listing executions for our pipeline...
Please wait...
Listing executions for our pipeline...
Please wait...
Listing executions for our pipeline...
Please wait...
Listing executions for our pipeline...
Please wait...
Listing executions for our pipeline...
Please wait...
Listing executions for our pipeline...
Please wait...
Listing executions for our pipeline...
Please wait...
Listing executions for our pipeline...
Please wait...
Listing executions for our pipeline...
Please wait...
Listing executions for our p

KeyboardInterrupt: 

In [37]:
build_pipeline_name = "sagemaker-{}-modelbuild".format(sagemaker_project_name_and_id)

In [38]:
from IPython.core.display import display, HTML

display(
    HTML(
        '<b>Check <a target="blank" href="https://console.aws.amazon.com/codesuite/codepipeline/pipelines/{}/view?region={}">ModelBuild</a> Pipeline</b>'.format(
            build_pipeline_name, region
        )
    )
)

# Wait For Pipeline Execution To Complete

In [39]:
%%time

import time
from pprint import pprint

executions_response = sm.list_pipeline_executions(PipelineName=sagemaker_project_name_and_id)[
    "PipelineExecutionSummaries"
]
pipeline_execution_status = executions_response[0]["PipelineExecutionStatus"]
print(pipeline_execution_status)

while pipeline_execution_status == "Executing":
    try:
        executions_response = sm.list_pipeline_executions(PipelineName=sagemaker_project_name_and_id)[
            "PipelineExecutionSummaries"
        ]
        pipeline_execution_status = executions_response[0]["PipelineExecutionStatus"]
    #        print('Executions for our pipeline...')
    #        print(pipeline_execution_status)
    except Exception as e:
        print("Please wait...")
        time.sleep(30)

pprint(executions_response)

ResourceNotFound: An error occurred (ResourceNotFound) when calling the ListPipelineExecutions operation: Pipeline 'arn:aws:sagemaker:us-east-1:231218423789:pipeline/dsoaws-1615231417-p-nynqfqk1cz5v' does not exist.

# _Wait for the Pipeline Execution ^^ Above ^^ to Complete_

# List Pipeline Execution Steps

In [40]:
pipeline_execution_status = executions_response[0]["PipelineExecutionStatus"]
print(pipeline_execution_status)

NameError: name 'executions_response' is not defined

In [ ]:
pipeline_execution_arn = executions_response[0]["PipelineExecutionArn"]
print(pipeline_execution_arn)

In [ ]:
pd.set_option("max_colwidth", 1000)

In [ ]:
from pprint import pprint

steps = sm.list_pipeline_execution_steps(PipelineExecutionArn=pipeline_execution_arn)

pprint(steps)

In [ ]:
import time
from sagemaker.lineage.visualizer import LineageTableVisualizer

viz = LineageTableVisualizer(sagemaker.session.Session())

for execution_step in reversed(steps["PipelineExecutionSteps"]):
    print(execution_step)
    # We are doing this because there appears to be a bug of this LineageTableVisualizer handling the Processing Step
    if execution_step["StepName"] == "Processing":
        processing_job_name = execution_step["Metadata"]["ProcessingJob"]["Arn"].split("/")[-1]
        print(processing_job_name)
        display(viz.show(processing_job_name=processing_job_name))
    elif execution_step["StepName"] == "Train":
        training_job_name = execution_step["Metadata"]["TrainingJob"]["Arn"].split("/")[-1]
        print(training_job_name)
        display(viz.show(training_job_name=training_job_name))
    else:
        display(viz.show(pipeline_execution_step=execution_step))
        time.sleep(5)

# Approve the Registered Model for Staging
The pipeline that was executed created a Model Package version within the specified Model Package Group. Of particular note, the registration of the model/creation of the Model Package was done so with approval status as `PendingManualApproval`.

Notes:  
* You can do this within SageMaker Studio, as well.  However, we are approving programmatically here in this example.
* This approval is only for Staging.  For Production, you must go through the CodePipeline (deep link is below.)


In [ ]:
%%time

import time
from pprint import pprint

while True:
    try:
        print("Executions for our pipeline...")
        list_model_packages_response = sm.list_model_packages(ModelPackageGroupName=sagemaker_project_name_and_id)
        break
    except Exception as e:
        print("Please wait...")
        time.sleep(30)

pprint(list_model_packages_response)

In [ ]:
time.sleep(30)

model_package_arn = list_model_packages_response["ModelPackageSummaryList"][0]["ModelPackageArn"]
print(model_package_arn)

In [ ]:
model_package_update_response = sm.update_model_package(
    ModelPackageArn=model_package_arn,
    ModelApprovalStatus="Approved",
)

print(model_package_update_response)

In [ ]:
time.sleep(30)

model_name = sm.list_models()["Models"][0]["ModelName"]
print(model_name)

In [ ]:
from IPython.core.display import display, HTML

display(
    HTML(
        '<b>Review <a target="blank" href="https://console.aws.amazon.com/sagemaker/home?region={}#/models/{}">Model</a></b>'.format(
            region, model_name
        )
    )
)

In [ ]:
deploy_pipeline_name = "sagemaker-{}-modeldeploy".format(sagemaker_project_name_and_id)

In [ ]:
from IPython.core.display import display, HTML

display(
    HTML(
        '<b>Check <a target="blank" href="https://console.aws.amazon.com/codesuite/codepipeline/pipelines/{}/view?region={}">ModelDeploy</a> Pipeline</b>'.format(
            deploy_pipeline_name, region
        )
    )
)

In [ ]:
staging_endpoint_name = "{}-staging".format(sagemaker_project_name)

In [ ]:
from IPython.core.display import display, HTML

display(
    HTML(
        '<b>Review <a target="blank" href="https://console.aws.amazon.com/sagemaker/home?region={}#/endpoints/{}">SageMaker Staging REST Endpoint</a></b>'.format(
            region, staging_endpoint_name
        )
    )
)

# _Wait Until the Staging Endpoint is Deployed_


In [ ]:
%%time

while True:
    try:
        waiter = sm.get_waiter("endpoint_in_service")
        print("Waiting for staging endpoint to be in `InService`...")
        waiter.wait(EndpointName=staging_endpoint_name)
        break
    except:
        print("Waiting for staging endpoint to be in `Creating`...")
        time.sleep(30)

print("Staging endpoint deployed.")

# _Wait Until the ^^ Staging Endpoint ^^ is Deployed_

# List Artifact Lineage

In [ ]:
from pprint import pprint

steps = sm.list_pipeline_execution_steps(PipelineExecutionArn=pipeline_execution_arn)

In [ ]:
import time
from sagemaker.lineage.visualizer import LineageTableVisualizer

viz = LineageTableVisualizer(sagemaker.session.Session())

for execution_step in reversed(steps["PipelineExecutionSteps"]):
    print(execution_step)
    # We are doing this because there appears to be a bug of this LineageTableVisualizer handling the Processing Step
    if execution_step["StepName"] == "Processing":
        processing_job_name = execution_step["Metadata"]["ProcessingJob"]["Arn"].split("/")[-1]
        print(processing_job_name)
        display(viz.show(processing_job_name=processing_job_name))
    elif execution_step["StepName"] == "Train":
        training_job_name = execution_step["Metadata"]["TrainingJob"]["Arn"].split("/")[-1]
        print(training_job_name)
        display(viz.show(training_job_name=training_job_name))
    else:
        display(viz.show(pipeline_execution_step=execution_step))
        time.sleep(5)

# Run A Sample Prediction in Staging

In [ ]:
import json
from sagemaker.tensorflow.model import TensorFlowPredictor
from sagemaker.serializers import JSONLinesSerializer
from sagemaker.deserializers import JSONLinesDeserializer

predictor = TensorFlowPredictor(
    endpoint_name=staging_endpoint_name,
    sagemaker_session=sess,
    model_name="saved_model",
    model_version=0,
    content_type="application/jsonlines",
    accept_type="application/jsonlines",
    serializer=JSONLinesSerializer(),
    deserializer=JSONLinesDeserializer(),
)

In [ ]:
inputs = [{"features": ["This is great!"]}, {"features": ["This is bad."]}]

predicted_classes = predictor.predict(inputs)

for predicted_class in predicted_classes:
    print("Predicted star_rating: {}".format(predicted_class))

# Deploy to Production

In [ ]:
from IPython.core.display import display, HTML

display(
    HTML(
        '<b>Review <a target="blank" href="https://console.aws.amazon.com/codesuite/codepipeline/pipelines/sagemaker-{}-modeldeploy/view?region={}"> Deploy to Production </a> Pipeline</b> '.format(
            sagemaker_project_name_and_id, region
        )
    )
)

In [ ]:
stage_name = "DeployStaging"
action_name = "ApproveDeployment"

In [ ]:
time.sleep(30)

stage_states = codepipeline.get_pipeline_state(name=deploy_pipeline_name)["stageStates"]

for stage_state in stage_states:

    if stage_state["stageName"] == stage_name:
        for action_state in stage_state["actionStates"]:
            if action_state["actionName"] == action_name:
                token = action_state["latestExecution"]["token"]

print(token)

In [ ]:
response = codepipeline.put_approval_result(
    pipelineName=deploy_pipeline_name,
    stageName=stage_name,
    actionName=action_name,
    result={"summary": "Approve from Staging to Production", "status": "Approved"},
    token=token,
)

# Get the Production Endpoint Name

In [ ]:
time.sleep(30)

production_endpoint_name = "{}-prod".format(sagemaker_project_name)

In [ ]:
from IPython.core.display import display, HTML

display(
    HTML(
        '<b>Review <a target="blank" href="https://console.aws.amazon.com/sagemaker/home?region={}#/endpoints/{}">SageMaker Production REST Endpoint</a></b>'.format(
            region, production_endpoint_name
        )
    )
)

# _Wait Until the Production Endpoint is Deployed_

In [ ]:
%%time

while True:
    try:
        waiter = sm.get_waiter("endpoint_in_service")
        print("Waiting for production endpoint to be in `InService`...")
        waiter.wait(EndpointName=production_endpoint_name)
        break
    except:
        print("Waiting for production endpoint to be in `Creating`...")
        time.sleep(30)

print("Production endpoint deployed.")

# _Wait Until the ^^ Production Endpoint ^^ is Deployed_

In [ ]:
import json
from sagemaker.tensorflow.model import TensorFlowPredictor
from sagemaker.serializers import JSONLinesSerializer
from sagemaker.deserializers import JSONLinesDeserializer

predictor = TensorFlowPredictor(
    endpoint_name=production_endpoint_name,
    sagemaker_session=sess,
    model_name="saved_model",
    model_version=0,
    content_type="application/jsonlines",
    accept_type="application/jsonlines",
    serializer=JSONLinesSerializer(),
    deserializer=JSONLinesDeserializer(),
)

In [ ]:
inputs = [{"features": ["This is great!"]}, {"features": ["This is bad."]}]

predicted_classes = predictor.predict(inputs)

for predicted_class in predicted_classes:
    print("Predicted star_rating: {}".format(predicted_class))

# List Artifact Lineage

In [ ]:
from pprint import pprint

steps = sm.list_pipeline_execution_steps(PipelineExecutionArn=pipeline_execution_arn)

In [ ]:
import time
from sagemaker.lineage.visualizer import LineageTableVisualizer

viz = LineageTableVisualizer(sagemaker.session.Session())

for execution_step in reversed(steps["PipelineExecutionSteps"]):
    print(execution_step)
    # We are doing this because there appears to be a bug of this LineageTableVisualizer handling the Processing Step
    if execution_step["StepName"] == "Processing":
        processing_job_name = execution_step["Metadata"]["ProcessingJob"]["Arn"].split("/")[-1]
        print(processing_job_name)
        display(viz.show(processing_job_name=processing_job_name))
    elif execution_step["StepName"] == "Train":
        training_job_name = execution_step["Metadata"]["TrainingJob"]["Arn"].split("/")[-1]
        print(training_job_name)
        display(viz.show(training_job_name=training_job_name))
    else:
        display(viz.show(pipeline_execution_step=execution_step))
        time.sleep(5)

# Release Resources

In [ ]:
%%html

<p><b>Shutting down your kernel for this notebook to release resources.</b></p>
<button class="sm-command-button" data-commandlinker-command="kernelmenu:shutdown" style="display:none;">Shutdown Kernel</button>
        
<script>
try {
    els = document.getElementsByClassName("sm-command-button");
    els[0].click();
}
catch(err) {
    // NoOp
}    
</script>

In [ ]:
%%javascript

try {
    Jupyter.notebook.save_checkpoint();
    Jupyter.notebook.session.delete();
}
catch(err) {
    // NoOp
}